In [48]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint 

In [8]:
mongo = MongoClient(port=27017)

In [9]:
print(mongo.list_database_names())

['Homework1', 'Project-3', 'admin', 'classDB', 'config', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [10]:
db = mongo['Project-3']

In [11]:
print(db.list_collection_names())

['video_games']


In [20]:
video_games = db['video_games']

In [13]:
print(db.video_games.find_one())

{'_id': ObjectId('65839489b3e6510dae7ccd06'), 'name': 'The Legend of Zelda: Ocarina of Time', 'platform': ' Nintendo 64', 'release_date': 'November 23, 1998', 'summary': 'As a young boy, Link is tricked by Ganondorf, the King of the Gerudo Thieves. The evil human uses Link to gain access to the Sacred Realm, where he places his tainted hands on Triforce and transforms the beautiful Hyrulean landscape into a barren wasteland. Link is determined to fix the problems he helped to create, so with the help of Rauru he travels through time gathering the powers of the Seven Sages.', 'user_review': 9.1}


In [43]:
# Delete documents with user_review equal to "tbd"
result = video_games.delete_many({'user_review': 'tbd'})
print(f"{result.deleted_count} documents deleted")

# Now, retrieve the remaining documents and perform aggregation
pipeline = [
    {
        '$group': {
            '_id': '$name',
            'mean_user_review': {'$avg': '$user_review'}
        }
    }
]

result_after_deletion = list(video_games.aggregate(pipeline))

for item in result_after_deletion:
    pprint(item)

0 documents deleted
{'_id': 'Control: Ultimate Edition', 'mean_user_review': 7.5}
{'_id': 'Yggdra Union', 'mean_user_review': 7.7}
{'_id': 'Kid Icarus: Uprising', 'mean_user_review': 8.7}
{'_id': 'Atelier Dusk Trilogy Deluxe Pack', 'mean_user_review': 6.2}
{'_id': 'Polybius', 'mean_user_review': 7.2}
{'_id': 'Persona 4: Dancing All Night', 'mean_user_review': 5.4}
{'_id': 'Behind the Frame: The Finest Scenery', 'mean_user_review': 8.6}
{'_id': 'Escape Plan', 'mean_user_review': 7.0}
{'_id': 'The Walking Dead: The Telltale Series - A New Frontier Episode 4: '
        'Thicker than Water',
 'mean_user_review': 5.550000000000001}
{'_id': 'Alter Ego (2010)', 'mean_user_review': 5.8}
{'_id': 'The Chaos Engine (2013)', 'mean_user_review': 7.6}
{'_id': 'Overwhelm', 'mean_user_review': 5.8}
{'_id': 'XCOM: Enemy Unknown Plus', 'mean_user_review': 8.1}
{'_id': "Clive Barker's Jericho", 'mean_user_review': 7.466666666666666}
{'_id': 'Top Gun: Combat Zones', 'mean_user_review': 4.8}
{'_id': 'Touho

In [46]:
documents_with_tbd = video_games.find({'user_review': 'tbd'})

# Print or process the matching documents
for document in documents_with_tbd:
    print(document)

In [54]:
cleaned_df = pd.DataFrame(result_after_deletion)
cleaned_df.head(15)

,_id,mean_user_review
0,Control: Ultimate Edition,7.500000
1,Yggdra Union,7.700000
2,Kid Icarus: Uprising,8.700000
3,Atelier Dusk Trilogy Deluxe Pack,6.200000
4,Polybius,7.200000
5,Persona 4: Dancing All Night,5.400000
6,Behind the Frame: The Finest Scenery,8.600000
7,Escape Plan,7.000000
8,The Walking Dead: The Telltale Series - A New ...,5.550000
9,Alter Ego (2010),5.800000


In [55]:
cleaned_df['mean_user_review'] = cleaned_df['mean_user_review'].round(2)

cleaned_df.head(15)

,_id,mean_user_review
0,Control: Ultimate Edition,7.50
1,Yggdra Union,7.70
2,Kid Icarus: Uprising,8.70
3,Atelier Dusk Trilogy Deluxe Pack,6.20
4,Polybius,7.20
5,Persona 4: Dancing All Night,5.40
6,Behind the Frame: The Finest Scenery,8.60
7,Escape Plan,7.00
8,The Walking Dead: The Telltale Series - A New ...,5.55
9,Alter Ego (2010),5.80
